<a href="https://colab.research.google.com/github/faruq2021/My-ML-MODEL-NOTEBOOKS/blob/main/Face_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi


Thu Dec  2 14:37:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Activation, BatchNormalization 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.metrics import categorical_crossentropy 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import os
import glob 
import itertools
import shutil
import random


In [5]:
os.chdir('/content/drive/MyDrive/me-vs-NotMe')
if os.path.isdir('train/faruq') is False:
  os.makedirs('train/faruq')
  os.makedirs('train/not_faruq')
  os.makedirs('valid/faruq')
  os.makedirs('valid/not_faruq')
  os.makedirs('test/faruq')
  os.makedirs('test/not_faruq')
  for c in random.sample(glob.glob('/faruq*'),15):
    shutil.move(c, 'train/faruq')
  for c in random.sample(glob.glob('/not_faruq*'),15):
    shutil.move(c, 'train/not_faruq')
  for c in random.sample(glob.glob('/faruq*'),5):
    shutil.move(c, 'valid/faruq')
  for c in random.sample(glob.glob('/not_faruq*'),5):
    shutil.move(c, 'valid/not_faruq')
  for c in random.sample(glob.glob('/faruq*'),3):
    shutil.move(c, 'test/faruq')
  for c in random.sample(glob.glob('/not_faruq*'),3):
    shutil.move(c, 'test/not_faruq')
os.chdir('../../')
 
  
  
  

In [6]:
#pathe= os.chdir('/content/drive/MyDrive/me-vs-NotMe')

In [7]:



#path = pathe
#faruq = [i for i in os.listdir(path) if os.path.isfile(os.path.join(path,i)) and 'faruq' in i]



In [8]:
#define parts to datasets 
train_path="/content/drive/MyDrive/me-vs-NotMe/train" 
test_path="/content/drive/MyDrive/me-vs-NotMe/test"
valid_path="/content/drive/MyDrive/me-vs-NotMe/valid"

In [18]:
#use keras image data generator to create process images
from keras.applications.vgg16 import preprocess_input
train_set= ImageDataGenerator(preprocessing_function=preprocess_input,zoom_range=[0.5,1.0],
horizontal_flip=True,brightness_range=[0.2,1.0],height_shift_range=0.5)\
.flow_from_directory(directory=train_path,target_size=(225,225),classes=['faruq','not_faruq'],batch_size=2)


Found 38 images belonging to 2 classes.


In [19]:
test_set= ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=test_path,target_size=(225,225),classes=['faruq','not_faruq'],batch_size=2)

Found 10 images belonging to 2 classes.


In [20]:
validation_set= ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=valid_path,target_size=(225,225),classes=['faruq','not_faruq'],batch_size=2)

Found 6 images belonging to 2 classes.


In [25]:
#print a batch of the training set
"""imgs, labels=next(train_set)
def plotimage (image_arr):
  fig, axes =plt.subplots(1,5,figsize=(20,20))
  axes=axes.flatten()
  for img, ax in zip(image_arr,axes):
    ax.imshow(img)
    ax.axis("off")
  plt.tight_layout()
  plt.show()"""


In [41]:
#build and train CNN 
fNet1=Sequential([Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(225,225,3)),
     MaxPool2D(pool_size=(2,2),strides=2),
     Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
     MaxPool2D(pool_size=(2,2),strides=2),
     Flatten(), Dense(units=2,activation='softmax')])
    

In [45]:
fNet1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 200704)            0         
                                                                 
 dense (Dense)               (None, 2)                 4

In [52]:
#compile model 
fNet1.compile(optimizer=Adam(learning_rate=0.00001),loss='BinaryCrossentropy',metrics=['Accuracy'])

In [55]:
fNet1.fit(x=train_set,validation_data=validation_set, epochs=25, verbose=2)

Epoch 1/25
19/19 - 1s - loss: 0.7477 - Accuracy: 0.7632 - val_loss: 0.9204 - val_Accuracy: 0.8333 - 1s/epoch - 54ms/step
Epoch 2/25
19/19 - 1s - loss: 0.4087 - Accuracy: 0.8684 - val_loss: 1.6912 - val_Accuracy: 0.6667 - 944ms/epoch - 50ms/step
Epoch 3/25
19/19 - 1s - loss: 0.6078 - Accuracy: 0.8947 - val_loss: 1.1218 - val_Accuracy: 0.6667 - 990ms/epoch - 52ms/step
Epoch 4/25
19/19 - 1s - loss: 0.4765 - Accuracy: 0.8684 - val_loss: 0.9788 - val_Accuracy: 0.6667 - 1s/epoch - 54ms/step
Epoch 5/25
19/19 - 1s - loss: 0.3922 - Accuracy: 0.9211 - val_loss: 1.3750 - val_Accuracy: 0.6667 - 950ms/epoch - 50ms/step
Epoch 6/25
19/19 - 1s - loss: 0.5999 - Accuracy: 0.8158 - val_loss: 0.6074 - val_Accuracy: 0.8333 - 953ms/epoch - 50ms/step
Epoch 7/25
19/19 - 1s - loss: 0.3718 - Accuracy: 0.8947 - val_loss: 0.0844 - val_Accuracy: 1.0000 - 1s/epoch - 53ms/step
Epoch 8/25
19/19 - 1s - loss: 0.2647 - Accuracy: 0.8684 - val_loss: 0.6748 - val_Accuracy: 0.8333 - 981ms/epoch - 52ms/step
Epoch 9/25
19/19 

In [58]:
prediction=fNet1.predict(x=test_set)

In [59]:
np.round(prediction)

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [60]:
test_set.classes

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

In [63]:
test_loss, test_acc = fNet1.evaluate(test_set)

5/5 [==============================] - 0s 20ms/step - loss: 0.1579 - Accuracy: 1.0000


In [70]:
fNet1.save('/content/drive/MyDrive'
    ,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
    save_traces=True,
)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/assets
